In [57]:
import os
import cv2
import time
from gtts import gTTS
from pydub import AudioSegment
from PIL import Image, ImageDraw, ImageFont

# Configure paths
FFMPEG_PATH = r'C:\Users\Hp\Downloads\ffmpeg-master-latest-win64-gpl-shared\bin\ffmpeg.exe'
FFPROBE_PATH = r'C:\Users\Hp\Downloads\ffmpeg-master-latest-win64-gpl-shared\bin\ffprobe.exe'
ASSETS_DIR = r'C:\Users\Hp\CodeReels\assets'
os.makedirs(ASSETS_DIR, exist_ok=True)

# Set ffmpeg paths for pydub
AudioSegment.ffmpeg = FFMPEG_PATH
AudioSegment.ffprobe = FFPROBE_PATH

def close_file_handles(filepath):
    """Attempt to close any open handles to a file"""
    try:
        if os.path.exists(filepath):
            os.close(os.open(filepath, os.O_WRONLY))
    except:
        pass

def safe_remove(filepath):
    """Safely remove a file with retries"""
    for _ in range(3):  # Try 3 times
        try:
            close_file_handles(filepath)
            if os.path.exists(filepath):
                os.remove(filepath)
            return True
        except Exception as e:
            print(f"Warning: Could not remove {filepath} - {e}")
            time.sleep(0.5)  # Wait before retrying
    return False

def generate_audio(text, audio_path):
    """Generate audio from text using gTTS"""
    try:
        safe_remove(audio_path)
        tts = gTTS(text=text, lang='en', slow=False)
        tts.save(audio_path)
        print(f"Audio generated: {audio_path}")
        return True
    except Exception as e:
        print(f"Error generating audio: {e}")
        return False

def create_image_with_text(text, image_path, bg_image_path=None):
    """Create image with text overlay"""
    try:
        safe_remove(image_path)
        
        if bg_image_path and os.path.exists(bg_image_path):
            img = Image.open(bg_image_path).convert('RGB')
        else:
            img = Image.new('RGB', (1080, 1920), color=(0, 0, 0))
        
        draw = ImageDraw.Draw(img)
        
        try:
            font = ImageFont.truetype("arial.ttf", 60)
        except:
            font = ImageFont.load_default()
        
        # Text wrapping logic remains the same as before
        # ... (use your existing text wrapping code)
        
        img.save(image_path)
        print(f"Image generated: {image_path}")
        return True
    except Exception as e:
        print(f"Error creating image: {e}")
        return False

def create_video(image_path, audio_path, video_path):
    """Create video from static image with audio duration"""
    try:
        safe_remove(video_path)
        audio = AudioSegment.from_file(audio_path)
        duration_sec = len(audio) / 1000
        
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError("Could not read image file")
        
        height, width = img.shape[:2]
        fps = 30
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))
        
        for _ in range(int(duration_sec * fps)):
            out.write(img)
        
        out.release()
        print(f"Video generated: {video_path}")
        return True
    except Exception as e:
        print(f"Error creating video: {e}")
        return False

def combine_video_audio(video_path, audio_path, output_path):
    """Combine video and audio using ffmpeg"""
    try:
        safe_remove(output_path)
        cmd = f'"{FFMPEG_PATH}" -y -i "{video_path}" -i "{audio_path}" -c:v copy -c:a aac -shortest "{output_path}"'
        os.system(cmd)
        print(f"Final video generated: {output_path}")
        return True
    except Exception as e:
        print(f"Error combining video/audio: {e}")
        return False

def generate_reel(text, bg_image_path=None):
    """Main function to generate video reel"""
    audio_path = os.path.join(ASSETS_DIR, "voice.mp3")
    image_path = os.path.join(ASSETS_DIR, "text_image.jpg")
    temp_video_path = os.path.join(ASSETS_DIR, "temp_video.mp4")
    final_output_path = os.path.join(ASSETS_DIR, "final_reel.mp4")
    
    # Clean up old files safely
    for path in [audio_path, image_path, temp_video_path, final_output_path]:
        safe_remove(path)
    
    # Execute pipeline with checks
    steps = [
        (generate_audio, (text, audio_path)),
        (create_image_with_text, (text, image_path, bg_image_path)),
        (create_video, (image_path, audio_path, temp_video_path)),
        (combine_video_audio, (temp_video_path, audio_path, final_output_path))
    ]
    
    for func, args in steps:
        if not func(*args):
            print(f"Failed at step: {func.__name__}")
            return False
    
    print("Video reel generation complete!")
    return True

if __name__ == "__main__":
    text_script = """Top-down parsing starts from the root and builds the tree downward using leftmost derivation.
    Bottom-up parsing works in reverse, starting from leaves and combining tokens until it reaches the start symbol."""
    
    background_path = os.path.join(ASSETS_DIR, "background.jpg") if os.path.exists(os.path.join(ASSETS_DIR, "background.jpg")) else None
    
    success = generate_reel(text_script, background_path)
    if success:
        print(f"Final video saved at: {os.path.join(ASSETS_DIR, 'final_reel.mp4')}")
    else:
        print("Failed to generate video reel")

Audio generated: C:\Users\Hp\CodeReels\assets\voice.mp3
Image generated: C:\Users\Hp\CodeReels\assets\text_image.jpg
Error creating video: [WinError 2] The system cannot find the file specified
Failed at step: create_video
Failed to generate video reel


C:\Users\Hp\Downloads\Lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


In [58]:
import os
import cv2
import time
from gtts import gTTS
from pydub import AudioSegment
from PIL import Image, ImageDraw, ImageFont
import subprocess

# Configure paths - UPDATE THESE TO YOUR ACTUAL PATHS
FFMPEG_PATH = r'C:\Users\Hp\Downloads\ffmpeg-master-latest-win64-gpl-shared\ffmpeg-master-latest-win64-gpl-shared\bin\ffmpeg.exe'  # Update this path
FFPROBE_PATH = r'C:\Users\Hp\Downloads\ffmpeg-master-latest-win64-gpl-shared\ffmpeg-master-latest-win64-gpl-shared\bin\ffprobe.exe'  # Update this path
ASSETS_DIR = r'C:\Users\Hp\CodeReels\assets'
os.makedirs(ASSETS_DIR, exist_ok=True)

# Set environment variables for pydub
os.environ['PATH'] += os.pathsep + os.path.dirname(FFMPEG_PATH)
os.environ['FFMPEG_PATH'] = FFMPEG_PATH
os.environ['FFPROBE_PATH'] = FFPROBE_PATH

def close_file_handles(filepath):
    """Helper function to close file handles on Windows"""
    try:
        if os.name == 'nt':  # Windows
            import ctypes
            kernel32 = ctypes.windll.kernel32
            handle = kernel32.CreateFileW(
                filepath, 0x80000000, 1, None, 3, 0x80, None)
            if handle != -1:
                kernel32.CloseHandle(handle)
    except:
        pass

def safe_remove(filepath):
    """Safely remove a file with retries and handle closing"""
    for _ in range(5):  # Try 5 times
        try:
            close_file_handles(filepath)
            if os.path.exists(filepath):
                os.remove(filepath)
                return True
        except Exception as e:
            print(f"Warning: Could not remove {filepath} - {e}")
            time.sleep(1)  # Wait longer between retries
    return False

def generate_audio(text, audio_path):
    """Generate audio from text using gTTS"""
    try:
        safe_remove(audio_path)
        tts = gTTS(text=text, lang='en', slow=False)
        tts.save(audio_path)
        print(f"Audio generated: {audio_path}")
        return True
    except Exception as e:
        print(f"Error generating audio: {e}")
        return False

def create_image_with_text(text, image_path, bg_image_path=None):
    """Create image with text overlay"""
    try:
        safe_remove(image_path)
        
        # Create background (1080x1920 for vertical video)
        if bg_image_path and os.path.exists(bg_image_path):
            img = Image.open(bg_image_path).convert('RGB')
            img = img.resize((1080, 1920))
        else:
            img = Image.new('RGB', (1080, 1920), color=(0, 0, 0))  # Black background
        
        draw = ImageDraw.Draw(img)
        
        # Try to load a nice font
        try:
            font = ImageFont.truetype("arial.ttf", 60)
        except:
            try:
                font = ImageFont.truetype("C:/Windows/Fonts/arial.ttf", 60)
            except:
                font = ImageFont.load_default(size=60)
        
        # Simple text wrapping
        lines = []
        words = text.split()
        current_line = []
        
        for word in words:
            test_line = ' '.join(current_line + [word])
            if draw.textlength(test_line, font=font) <= 1000:  # 1000px width
                current_line.append(word)
            else:
                lines.append(' '.join(current_line))
                current_line = [word]
        lines.append(' '.join(current_line))
        
        # Calculate positions
        y = (1920 - (len(lines) * 70)) // 2  # Approximate line height
        
        # Draw each line
        for line in lines:
            text_width = draw.textlength(line, font=font)
            x = (1080 - text_width) // 2
            draw.text((x, y), line, font=font, fill=(255, 255, 255))
            y += 70  # Line height
        
        img.save(image_path)
        print(f"Image generated: {image_path}")
        return True
    except Exception as e:
        print(f"Error creating image: {e}")
        return False

def create_video(image_path, audio_path, video_path):
    """Create video from static image with audio duration"""
    try:
        safe_remove(video_path)
        
        # Get audio duration using pydub
        try:
            audio = AudioSegment.from_file(audio_path)
            duration = len(audio) / 1000  # Convert to seconds
        except Exception as e:
            print(f"Error reading audio: {e}")
            duration = 5  # Default duration if audio fails
        
        # Create video using FFmpeg directly (more reliable)
        cmd = [
            FFMPEG_PATH,
            '-loop', '1',
            '-i', image_path,
            '-i', audio_path,
            '-c:v', 'libx264',
            '-t', str(duration),
            '-pix_fmt', 'yuv420p',
            '-shortest',
            video_path
        ]
        
        subprocess.run(cmd, check=True)
        print(f"Video generated: {video_path}")
        return True
    except Exception as e:
        print(f"Error creating video: {e}")
        return False

def generate_reel(text, bg_image_path=None):
    """Main function to generate video reel"""
    # Define paths
    audio_path = os.path.join(ASSETS_DIR, "voice.mp3")
    image_path = os.path.join(ASSETS_DIR, "text_image.jpg")
    video_path = os.path.join(ASSETS_DIR, "final_reel.mp4")
    
    # Clean up old files
    for path in [audio_path, image_path, video_path]:
        safe_remove(path)
    
    # Execute pipeline
    if not generate_audio(text, audio_path):
        return False
    
    if not create_image_with_text(text, image_path, bg_image_path):
        return False
    
    if not create_video(image_path, audio_path, video_path):
        return False
    
    print("Video reel generation complete!")
    return True

if __name__ == "__main__":
    # Example text
    text_script = """Top-down parsing starts from the root and builds the tree downward using leftmost derivation.
    Bottom-up parsing works in reverse, starting from leaves and combining tokens until it reaches the start symbol."""
    
    # Optional background image
    background_path = os.path.join(ASSETS_DIR, "background.jpg") if os.path.exists(os.path.join(ASSETS_DIR, "background.jpg")) else None
    
    # Generate the reel
    if generate_reel(text_script, background_path):
        print(f"Final video saved at: {os.path.join(ASSETS_DIR, 'final_reel.mp4')}")
    else:
        print("Failed to generate video reel")

Audio generated: C:\Users\Hp\CodeReels\assets\voice.mp3
Image generated: C:\Users\Hp\CodeReels\assets\text_image.jpg
Video generated: C:\Users\Hp\CodeReels\assets\final_reel.mp4
Video reel generation complete!
Final video saved at: C:\Users\Hp\CodeReels\assets\final_reel.mp4


In [51]:
from pydub import AudioSegment

# Set the path to ffmpeg and ffprobe manually
AudioSegment.ffmpeg = r'C:\Users\Hp\Downloads\ffmpeg-master-latest-win64-gpl-shared\ffmpeg-master-latest-win64-gpl-shared\bin\ffmpeg.exe'
AudioSegment.ffprobe = r'C:\Users\Hp\Downloads\ffmpeg-master-latest-win64-gpl-shared\ffmpeg-master-latest-win64-gpl-shared\bin\ffprobe.exe'

# Now you can proceed with the rest of your code as before


In [64]:
import os
import cv2
import time
import numpy as np
from gtts import gTTS
from pydub import AudioSegment
from PIL import Image, ImageDraw, ImageFont
import subprocess

# Configure paths
FFMPEG_PATH = r'C:\Users\Hp\Downloads\ffmpeg-master-latest-win64-gpl-shared\bin\ffmpeg.exe'
FFPROBE_PATH = r'C:\Users\Hp\Downloads\ffmpeg-master-latest-win64-gpl-shared\bin\ffprobe.exe'
ASSETS_DIR = r'C:\Users\Hp\CodeReels\assets'
os.makedirs(ASSETS_DIR, exist_ok=True)

# Set ffmpeg paths for pydub
AudioSegment.ffmpeg = FFMPEG_PATH
AudioSegment.ffprobe = FFPROBE_PATH

def close_file_handles(filepath):
    """Attempt to close any open handles to a file"""
    try:
        if os.path.exists(filepath):
            os.close(os.open(filepath, os.O_WRONLY))
    except:
        pass

def safe_remove(filepath):
    """Safely remove a file with retries"""
    for _ in range(3):
        try:
            close_file_handles(filepath)
            if os.path.exists(filepath):
                os.remove(filepath)
            return True
        except Exception as e:
            print(f"Warning: Could not remove {filepath} - {e}")
            time.sleep(0.5)
    return False

def generate_audio(text, audio_path):
    """Generate audio from text using gTTS"""
    try:
        safe_remove(audio_path)
        tts = gTTS(text=text, lang='en', slow=False)
        tts.save(audio_path)
        print(f"Audio generated: {audio_path}")
        return True
    except Exception as e:
        print(f"Error generating audio: {e}")
        return False

def wrap_text(text, font, max_width):
    """Wrap text to fit within specified width"""
    lines = []
    for paragraph in text.split('\n'):
        words = paragraph.split()
        current_line = []
        
        for word in words:
            test_line = ' '.join(current_line + [word])
            test_width = font.getlength(test_line)
            
            if test_width <= max_width:
                current_line.append(word)
            else:
                if current_line:
                    lines.append(' '.join(current_line))
                current_line = [word]
        
        if current_line:
            lines.append(' '.join(current_line))
    
    return lines

def create_animated_text_frame(draw, img, text_lines, font, progress, frame_num):
    """Create animated text frame with various effects"""
    width, height = img.size
    total_lines = len(text_lines)
    
    # Base color (white with rainbow effect)
    base_color = (
        int(255 * (0.5 + 0.5 * np.sin(progress * 2 * np.pi + 0))),
        int(255 * (0.5 + 0.5 * np.sin(progress * 2 * np.pi + 2))),
        int(255 * (0.5 + 0.5 * np.sin(progress * 2 * np.pi + 4))))
    
    
    # Draw each line with animation
    for i, line in enumerate(text_lines):
        # Calculate vertical position with wave effect
        line_progress = (progress + i * 0.1) % 1.0
        y_offset = int(20 * np.sin(line_progress * 2 * np.pi))
        
        # Calculate horizontal position
        text_width = font.getlength(line)
        x = (width - text_width) // 2
        y = (height - total_lines * 70) // 2 + i * 70 + y_offset
        
        # Draw text with outline
        border_width = 2
        for dx in [-border_width, 0, border_width]:
            for dy in [-border_width, 0, border_width]:
                if dx != 0 or dy != 0:
                    draw.text((x+dx, y+dy), line, fill=(0,0,0), font=font)
        
        # Draw main text with color that changes per line
        line_color = (
            int(base_color[0] * (0.7 + 0.3 * np.sin(i * 0.5))),
            int(base_color[1] * (0.7 + 0.3 * np.sin(i * 0.5 + 2))),
            int(base_color[2] * (0.7 + 0.3 * np.sin(i * 0.5 + 4))))
        
        draw.text((x, y), line, fill=line_color, font=font)

def create_animated_image(text, image_path, bg_image_path=None, duration=5):
    """Create animated image with text effects"""
    try:
        safe_remove(image_path)
        
        if bg_image_path and os.path.exists(bg_image_path):
            img = Image.open(bg_image_path).convert('RGB')
            img = img.resize((1080, 1920))
        else:
            # Create gradient background
            img = Image.new('RGB', (1080, 1920), color=(0, 0, 0))
            draw = ImageDraw.Draw(img)
            for y in range(1920):
                gradient = int(255 * y / 1920)
                draw.line([(0, y), (1080, y)], fill=(gradient//4, gradient//2, gradient))
        
        try:
            font = ImageFont.truetype("arial.ttf", 60)
        except:
            font = ImageFont.load_default(size=60)
        
        # Wrap text
        max_width = 1000  # Allow 40px margin on each side
        text_lines = wrap_text(text, font, max_width)
        
        # Generate frames
        fps = 30
        num_frames = int(duration * fps)
        
        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(image_path, fourcc, fps, (1080, 1920))
        
        for frame_num in range(num_frames):
            frame = img.copy()
            draw = ImageDraw.Draw(frame)
            progress = frame_num / num_frames
            
            create_animated_text_frame(draw, frame, text_lines, font, progress, frame_num)
            
            # Convert PIL Image to numpy array for OpenCV
            frame_cv = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)
            out.write(frame_cv)
        
        out.release()
        print(f"Animated image saved as video: {image_path}")
        return True
    except Exception as e:
        print(f"Error creating animated image: {e}")
        return False

def create_video_with_audio(image_path, audio_path, output_path):
    """Combine animated video with audio"""
    try:
        safe_remove(output_path)
        
        # Get audio duration
        audio = AudioSegment.from_file(audio_path)
        audio_duration = len(audio) / 1000
        
        # Use FFmpeg to combine
        cmd = [
            FFMPEG_PATH,
            '-y',
            '-i', image_path,
            '-i', audio_path,
            '-c:v', 'libx264',
            '-c:a', 'aac',
            '-strict', 'experimental',
            '-shortest',
            '-pix_fmt', 'yuv420p',
            output_path
        ]
        
        subprocess.run(cmd, check=True)
        print(f"Final video with animations saved: {output_path}")
        return True
    except Exception as e:
        print(f"Error creating final video: {e}")
        return False

def generate_reel(text, bg_image_path=None):
    """Main function to generate animated video reel"""
    audio_path = os.path.join(ASSETS_DIR, "voice.mp3")
    animated_image_path = os.path.join(ASSETS_DIR, "animated_text.mp4")
    final_output_path = os.path.join(ASSETS_DIR, "final_reel.mp4")
    
    # Clean up old files
    for path in [audio_path, animated_image_path, final_output_path]:
        safe_remove(path)
    
    # Execute pipeline
    if not generate_audio(text, audio_path):
        return False
    
    if not create_animated_image(text, animated_image_path, bg_image_path):
        return False
    
    if not create_video_with_audio(animated_image_path, audio_path, final_output_path):
        return False
    
    print("Animated video reel generation complete!")
    return True

if __name__ == "__main__":
    text_script = """Top-down parsing starts from the root and builds the tree downward using leftmost derivation.
Bottom-up parsing works in reverse, starting from leaves and combining tokens until it reaches the start symbol."""
    
    background_path = os.path.join(ASSETS_DIR, "background.jpg") if os.path.exists(os.path.join(ASSETS_DIR, "background.jpg")) else None
    
    success = generate_reel(text_script, background_path)
    if success:
        print(f"Final animated video saved at: {os.path.join(ASSETS_DIR, 'final_reel.mp4')}")
    else:
        print("Failed to generate animated video reel")

Audio generated: C:\Users\Hp\CodeReels\assets\voice.mp3
Error creating animated image: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Failed to generate animated video reel
